# Data Preprocessing

We need to import and transform following MIMIC-III datasets:
DIAGNOSES_ICD.csv: Each row in this file maps Hospitalization ID (HADM_ID) of a patient with a unique ICD_9 CODE.
  
  Example:
  
  |ROW_ID|SUBJECT_ID|**HADM_ID**|SEQ_NUM|**ICD9_CODE**|
  |--------|------------|---------|---------|-----------|
  |1297|109|172335|1|"0030"|
  |1299|109|172335|3|"0038"|
  |1301|109|173633|5|"0031"|
  
  
  The aim is to transform this dataset to dataframe DIGNOSES with index as HADM_ID and columns as unique ICD_9 codes (6984 in total), to represent multi-hot encoding of ICD_9 codes for given hospitalization.
  
  |HADM_ID|ICD9_CODE_0030|ICD9_CODE_0031|ICD9_CODE_0038|
  |-------|--------------|--------------|--------------|
  |172335|1|0|1|
  |173633|0|1|0|
  
  
  
  NOTEEVENTS.csv: Each row maps HADM_ID (Hospitalization ID) with a free text Discharge summary (TEXT) field.
  
  |ROW_ID|SUBJECT_ID|**HADM_ID**|CHARTDATE|CHARTTIME|STORETIME|CATEGORY|DESCRIPTION|CGID|ISERROR|**TEXT**|
  |------|----------|-----------|---------|---------|---------|--------|-----------|----|-------|--------|
  |174|22532|167853|2151-08-04|||Discharge summary|Report|||Admission Date:  [\*\*2151-7-16**]       Discharge Date:  [\*\*2151-8-4**] Service: ADDENDUM: RADIOLOGIC STUDIES:  Radiologic studies also included a chest| 
 
  The aim is to transform this to dataframe with HADM_ID as index and TEXT as column.

  |HADM_ID|TEXT|
  |-------|----|
  |167853|Admission Date:  [\*\*2151-7-16**]       Discharge Date:  [\*\*2151-8-4**] Service: ADDENDUM: RADIOLOGIC STUDIES:  Radiologic studies also included a chest|

<br/>
<br/>

The processed dataframes are then stored (as csv) for further usage.

## Critical Imports

In [ ]:
# Critical imports
import os
import numpy as np
import pandas as pd
import time

In [ ]:
# Mount the project directory in Google drive. (Its only intended to be run in colab environment.)

from google.colab import drive
drive.mount('drive')

In [ ]:
# Define the base project directory.

PROJECT_DIR = 'drive/My Drive/cs598-dl/' # For Google drive only

# PROJECT_DIR = '../' # For local directory

In [ ]:
'''We first process MIMIC-III DIAGNOSES_ICD dataset.'''

In [ ]:
sta = time.time()

In [ ]:
# Read DIAGNOSES_ICD.csv from data directory, and pre-process.

diagnoses_df = pd.read_csv(PROJECT_DIR + 'data/DIAGNOSES_ICD.csv', usecols=['HADM_ID', 'ICD9_CODE'])
diagnoses_df = diagnoses_df.astype({'ICD9_CODE': 'string'})

# Collect all unique ICD_9 codes and create new DataFrame codes_df.
codes_df = pd.DataFrame(diagnoses_df['ICD9_CODE'].unique(), columns = ['ICD9_CODE'])

# Create DataFrame representing one-hot encoding of ICD_9 codes.
one_hot_enc_df = pd.get_dummies(codes_df, columns = ['ICD9_CODE'], dtype='bool')

# Join codes_df and one_hot_enc_df, based on index
codes_df = codes_df.join(one_hot_enc_df)

# Merge diagnoses_df and codes_df based on column 'ICD9_CODE'. 
# Dask operations are lazy and do not materialize until 'compute()' method is invoked.
diagnoses_df = diagnoses_df.merge(codes_df, on='ICD9_CODE')

diagnoses_df = diagnoses_df.drop(['ICD9_CODE'], axis = 1)

# This step will group all ICD_9 codes corresponding to a given HADM_ID and build a multi-hot embedding.
diagnoses_df = diagnoses_df.groupby('HADM_ID').any().reset_index()

print(diagnoses_df)

In [ ]:


# Read DIAGNOSES_ICD.csv from data directory, and pre-process.

diagnoses_df = pd.read_csv(PROJECT_DIR + 'data/DIAGNOSES_ICD.csv', usecols=['HADM_ID', 'ICD9_CODE'])
diagnoses_df = diagnoses_df.astype({'ICD9_CODE': 'string'})

# Collect all unique ICD_9 codes and create new DataFrame codes_df.
codes_df = pd.DataFrame(diagnoses_df['ICD9_CODE'].unique(), columns = ['ICD9_CODE'])

# Create DataFrame representing one-hot encoding of ICD_9 codes.
one_hot_enc_df = pd.get_dummies(codes_df, columns = ['ICD9_CODE'], dtype='bool')

# Join codes_df and one_hot_enc_df, based on index
codes_df = codes_df.join(one_hot_enc_df)

# Merge diagnoses_df and codes_df based on column 'ICD9_CODE'. 
diagnoses_df = diagnoses_df.merge(codes_df, on='ICD9_CODE')
diagnoses_df = diagnoses_df.drop(['ICD9_CODE'], axis = 1)

# This step will group all ICD_9 codes corresponding to a given HADM_ID and build a multi-hot embedding.
diagnoses_df = diagnoses_df.groupby('HADM_ID').any().reset_index()

print(diagnoses_df)

In [ ]:
'''Next we process MIMIC-III NOTEEVENTS dataset.'''

In [ ]:
# Import dataset and pre-process.
notes_df = pd.read_csv('drive/My Drive/cs598-dl/data/NOTEEVENTS.csv', usecols=['HADM_ID', "CATEGORY","DESCRIPTION", "TEXT"])
notes_df = notes_df.dropna()

# Only filter-in notes which are 'Discharge summary' and are of sub-type 'Report'.
notes_df = notes_df[(notes_df['CATEGORY'] == 'Discharge summary') & (notes_df['DESCRIPTION'] == 'Report')]
notes_df = notes_df.drop(['CATEGORY', 'DESCRIPTION'], axis=1)
notes_df = notes_df.astype({'HADM_ID': 'int64'})
notes_df = notes_df.drop_duplicates(subset = 'HADM_ID')
print(notes_df)

In [ ]:
'''
We next select the subset of rows in diagnoses_df and notes_df with common set of HADM_IDs, 
and remove other rows from each DataFrame. Such rows can not be used in training or testing.
'''

In [ ]:
# Collect all hadm_ids from diagnoses_df
hadm_ids_from_diagnoses_df = diagnoses_df.filter(items = ['HADM_ID'])

# Collect all hadm_ids from diagnoses_df
hadm_ids_from_notes_df = notes_df.filter(items = ['HADM_ID'])

# Generate DataFrame with common set of HADM_IDs.
hadm_ids_df = hadm_ids_from_diagnoses_df.merge(hadm_ids_from_notes_df, how = 'inner')

# Filter rows in daignoses_df by merging with DataFrame containing common HADM_IDs.
diagnoses_df = diagnoses_df.merge(hadm_ids_df, on='HADM_ID', how = 'right')

# Similarly, filter rows in notes_df by merging with DataFrame containing common HADM_IDs.
notes_df = notes_df.merge(hadm_ids_df, on='HADM_ID', how = 'inner')

In [ ]:
# Pickle diagnoses_df
diagnoses_df.to_pickle(PROJECT_DIR + 'data/DIAGNOSES.pkl')

# Pickle notes_df
notes_df.to_pickle(PROJECT_DIR + 'data/NOTES.pkl')

In [ ]:
end = time.time()
print('Time taken in data processing:' + str(end - sta))